In [1]:
pip install fasttext

Processing /Users/anastasiamoiseva/Library/Caches/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3/fasttext-0.9.1-cp37-cp37m-macosx_10_7_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np 
import pandas as pd 
import fasttext
import bz2
import csv
from sklearn.metrics import roc_auc_score
import os

In [12]:
data = bz2.BZ2File("amazonreviews/train.ft.txt.bz2")
data = data.readlines()
data = [x.decode('utf-8') for x in data]
print(len(data))

3600000


In [6]:
data = pd.DataFrame(data)
data.to_csv("train.txt", index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [7]:
model = fasttext.train_supervised('train.txt',label_prefix='__label__', thread=4, epoch = 10)
print(model.labels, 'are the labels or targets the model is predicting')

['__label__1', '__label__2'] are the labels or targets the model is predicting


In [8]:
test = bz2.BZ2File("amazonreviews/test.ft.txt.bz2")
test = test.readlines()
test = [x.decode('utf-8') for x in test]
print(len(test), 'number of records in the test set') 

400000 number of records in the test set


In [9]:
new = [w.replace('__label__2 ', '') for w in test]
new = [w.replace('__label__1 ', '') for w in new]
new = [w.replace('\n', '') for w in new]

In [10]:
# Use the predict function 
pred = model.predict(new)

# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

['__label__2'] is the predicted label
['__label__2'] is the probability score


In [11]:
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))

0.9173074999999999


Now building 'hard baseline' model

In [13]:
train_labels  = [0 if x.split(' ')[0] == '__label__1' else 1 for x in data]

In [14]:
labels_tr = pd.DataFrame(train_labels)

In [15]:
new_train = [w.replace('__label__2 ', '') for w in data]
new_train = [w.replace('__label__1 ', '') for w in new_train]
new_train = [w.replace('\n', '') for w in new_train]

In [16]:
data_train = pd.DataFrame(new_train, dtype=str)
data_train[:4]

,0
0,Stuning even for the non-gamer: This sound tra...
1,The best soundtrack ever to anything.: I'm rea...
2,Amazing!: This soundtrack is my favorite music...
3,Excellent Soundtrack: I truly like this soundt...


In [17]:
data_train['Labels'] = labels_tr

In [18]:
data_train.columns = ['Message', 'Labels']

In [19]:
data_train.head()



,Message,Labels
0,Stuning even for the non-gamer: This sound tra...,1
1,The best soundtrack ever to anything.: I'm rea...,1
2,Amazing!: This soundtrack is my favorite music...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1


In [20]:
data_train.shape

(3600000, 2)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix

In [22]:
bow_transformer = CountVectorizer(lowercase=False).fit(data_train['Message'])
print(len(bow_transformer.vocabulary_))

1264355


In [23]:
amazon_bow = bow_transformer.transform(data_train['Message'])

In [25]:
print('Shape of Sparse Matrix: ',amazon_bow.shape)
print('Amount of non-zero occurences:',amazon_bow.nnz)

Shape of Sparse Matrix:  (3600000, 1264355)
Amount of non-zero occurences: 204911651


In [26]:
message4=data_train['Message'][3]
print(message4)

Excellent Soundtrack: I truly like this soundtrack and I enjoy video game music. I have played this game and most of the music on here I enjoy and it's truly relaxing and peaceful.On disk one. my favorites are Scars Of Time, Between Life and Death, Forest Of Illusion, Fortress of Ancient Dragons, Lost Fragment, and Drowned Valley.Disk Two: The Draggons, Galdorb - Home, Chronomantique, Prisoners of Fate, Gale, and my girlfriend likes ZelbessDisk Three: The best of the three. Garden Of God, Chronopolis, Fates, Jellyfish sea, Burning Orphange, Dragon's Prayer, Tower Of Stars, Dragon God, and Radical Dreamers - Unstealable Jewel.Overall, this is a excellent soundtrack and should be brought by those that like video game music.Xander Cross


In [27]:
bow4=bow_transformer.transform([message4])
print(bow4)
print(bow4.shape)

  (0, 70001)	1
  (0, 104927)	1
  (0, 122549)	1
  (0, 151457)	1
  (0, 151465)	1
  (0, 167472)	1
  (0, 185011)	1
  (0, 194704)	1
  (0, 199395)	1
  (0, 199418)	2
  (0, 199558)	1
  (0, 200041)	1
  (0, 200896)	1
  (0, 223570)	1
  (0, 234956)	1
  (0, 234975)	1
  (0, 243168)	1
  (0, 243694)	1
  (0, 244459)	1
  (0, 255504)	1
  (0, 255508)	1
  (0, 256823)	1
  (0, 264073)	2
  (0, 293778)	1
  (0, 306195)	1
  :	:
  (0, 896719)	1
  (0, 900906)	1
  (0, 935915)	1
  (0, 937564)	1
  (0, 961172)	2
  (0, 961703)	1
  (0, 997029)	1
  (0, 1001709)	3
  (0, 1002930)	2
  (0, 1020291)	4
  (0, 1024370)	1
  (0, 1024829)	1
  (0, 1043102)	1
  (0, 1054954)	1
  (0, 1095621)	1
  (0, 1122717)	1
  (0, 1133296)	1
  (0, 1149524)	2
  (0, 1184858)	1
  (0, 1185577)	2
  (0, 1192950)	3
  (0, 1194092)	1
  (0, 1194797)	1
  (0, 1209399)	2
  (0, 1233257)	2
(1, 1264355)


In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(amazon_bow)
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 1233257)	0.11495628469406777
  (0, 1209399)	0.12205180647557985
  (0, 1194797)	0.05568097520913043
  (0, 1194092)	0.04850073471213825
  (0, 1192950)	0.05191863672512791
  (0, 1185577)	0.027179581976913426
  (0, 1184858)	0.021071300466346456
  (0, 1149524)	0.14792546732506243
  (0, 1133296)	0.04326771361360803
  (0, 1122717)	0.08943493761960164
  (0, 1095621)	0.08745136115996081
  (0, 1054954)	0.061674341911787715
  (0, 1043102)	0.10010047914326554
  (0, 1024829)	0.02770588033091996
  (0, 1024370)	0.023456027973277806
  (0, 1020291)	0.06640872348129513
  (0, 1002930)	0.052291730762959204
  (0, 1001709)	0.13654932347800838
  (0, 997029)	0.04311651331496057
  (0, 961703)	0.069944501391638
  (0, 961172)	0.06027376245033363
  (0, 937564)	0.017195501411388998
  (0, 935915)	0.016977096319412726
  (0, 900906)	0.04935375243862997
  (0, 896719)	0.024052588067172587
  :	:
  (0, 306195)	0.12136618478812801
  (0, 293778)	0.08058048786774226
  (0, 264073)	0.13048746482527085
  (0, 256823)	0.09

In [29]:
amazon_tfidf=tfidf_transformer.transform(amazon_bow)
print(amazon_tfidf.shape)

(3600000, 1264355)


In [30]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(amazon_tfidf,data_train['Labels'])

In [31]:
all_predictions = spam_detect_model.predict(amazon_tfidf)
print(all_predictions)

[1 1 1 ... 1 1 1]


In [32]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(data_train['Labels'],all_predictions))
print(confusion_matrix(data_train['Labels'],all_predictions))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87   1800000
           1       0.88      0.85      0.87   1800000

    accuracy                           0.87   3600000
   macro avg       0.87      0.87      0.87   3600000
weighted avg       0.87      0.87      0.87   3600000

[[1590067  209933]
 [ 265192 1534808]]


In [33]:
print(roc_auc_score(data_train['Labels'], all_predictions))

0.8680208333333334


Now moving forward to neural model

In [34]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords




In [35]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [36]:
from keras.preprocessing.text import Tokenizer

In [35]:
data_train.head()

,Message,Labels
0,Great CD: My lovely Pat has one of the GREAT v...,1
1,One of the best game music soundtracks - for a...,1
2,Batteries died within a year ...: I bought thi...,1
3,"works fine, but Maha Energy is better: Check o...",1
4,Great for the non-audiophile: Reviewed quite a...,1


In [37]:
y_train = data_train['Labels']

In [38]:
data_test = pd.DataFrame(new)

In [39]:
data_test.head()

,0
0,Great CD: My lovely Pat has one of the GREAT v...
1,One of the best game music soundtracks - for a...
2,Batteries died within a year ...: I bought thi...
3,"works fine, but Maha Energy is better: Check o..."
4,Great for the non-audiophile: Reviewed quite a...


In [40]:
data_test.columns = ['Message']

In [41]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [42]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [43]:
X_train = []
sentences = list(data_train['Message'])
for sen in sentences:
    X_train.append(preprocess_text(sen))

In [44]:
X_train[3]

'Excellent Soundtrack truly like this soundtrack and enjoy video game music have played this game and most of the music on here enjoy and it truly relaxing and peaceful On disk one my favorites are Scars Of Time Between Life and Death Forest Of Illusion Fortress of Ancient Dragons Lost Fragment and Drowned Valley Disk Two The Draggons Galdorb Home Chronomantique Prisoners of Fate Gale and my girlfriend likes ZelbessDisk Three The best of the three Garden Of God Chronopolis Fates Jellyfish sea Burning Orphange Dragon Prayer Tower Of Stars Dragon God and Radical Dreamers Unstealable Jewel Overall this is excellent soundtrack and should be brought by those that like video game music Xander Cross'

In [45]:
X_test = []
sentences = list(data_test['Message'])
for sen in sentences:
    X_test.append(preprocess_text(sen))

In [46]:
data_test['Message'][3]

"works fine, but Maha Energy is better: Check out Maha Energy's website. Their Powerex MH-C204F charger works in 100 minutes for rapid charge, with option for slower charge (better for batteries). And they have 2200 mAh batteries."

In [48]:
X_test[3]

'works fine but Maha Energy is better Check out Maha Energy website Their Powerex MH F charger works in minutes for rapid charge with option for slower charge better for batteries And they have mAh batteries '

In [51]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [54]:
vocab_size

1103004

In [53]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [55]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [56]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [57]:
from keras import layers

In [58]:
from keras import optimizers

In [77]:
opt = optimizers.adam(lr=0.001)

In [59]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=True)
model.add(embedding_layer)


model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [68]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          110300400 
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 10001     
Total params: 110,310,401
Trainable params: 110,310,401
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
history = model.fit(X_train, y_train, batch_size=2500, epochs=5, verbose=1, validation_split=0.2)

//anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2880000 samples, validate on 720000 samples
Epoch 1/5
2880000/2880000 [==============================] - 4451s 2ms/step - loss: 14.7119 - acc: 0.8447 - val_loss: 17.2943 - val_acc: 0.8548
Epoch 2/5
2880000/2880000 [==============================] - 4733s 2ms/step - loss: 16.5583 - acc: 0.8552 - val_loss: 16.0924 - val_acc: 0.8557
Epoch 3/5
2880000/2880000 [==============================] - 4823s 2ms/step - loss: 16.4399 - acc: 0.8566 - val_loss: 16.8225 - val_acc: 0.8569
Epoch 4/5
2880000/2880000 [==============================] - 4587s 2ms/step - loss: 16.7710 - acc: 0.8569 - val_loss: 16.5355 - val_acc: 0.8602
Epoch 5/5
2880000/2880000 [==============================] - 4524s 2ms/step - loss: 16.9367 - acc: 0.8574 - val_loss: 16.9814 - val_acc: 0.8559


In [70]:
score = model.evaluate(X_test, labels, verbose=1)

400000/400000 [==============================] - 21s 54us/step


In [0]:
from keras.regularizers import l2

In [71]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 17.234908778186913
Test Accuracy: 0.8543199896812439


In [72]:
all_predictions1 = model.predict(X_test)

In [73]:
print(roc_auc_score(labels, all_predictions1))

0.8671587674749999


Okey, so roc is lower but most porobably it due to the too large learnigng rate and small number of epochs (but sadly enough my laptop cannot process more than that and google colab stops session automatically)

I believe that given the roc score (almost the same as for hard baseline model) that was produced by simple NN-model with 6 epochs, with increased number of epochs (and more complex structure), it will out-perform hard baseline model.

Because of a potential drawback with one-hot encoded feature vector approaches such as N-Grams, bag of words and TF-IDF approach being that the feature vector for each document can be huge, the NN model, which does not encounter the same problem, should perform better

**Conclusion:**

1. The best model so far is fasttext
2. NN showed not bad results, but to make it better: – 1) increase # of epochs and change structure to LSTM (as it is a powerful and popular tool to work with sequential data like text or time series); 2) make learning rate smaller